## Part 1 - Organize the Audio Data

In [1]:
import utils as u
import librosa
import soundfile as sf
import os
import numpy as np

#### Section for getting the bonafide files

In [ ]:
import sys
if len(sys.argv) != 4:
    print("Usage: python main.py <dataset_directory> <timestamp_file> <output_directory>")
    sys.exit(1)

dataset_directory = sys.argv[1]
timestamp_file = sys.argv[2]
output_directory = sys.argv[3]

u.process_dataset(dataset_directory, timestamp_file, output_directory)

In [ ]:
# no need to convert again in the future, just use the files in wav_files folder directly
#data_home = '/Volumes/T7/group5_DL4M/dev_set'
#u.convert_flac_to_wav_librosa(data_home)

In [2]:
wav_directory = '/Users/mona/Documents/GitHub/group_5_DL4M1/new_data/japanese_full_traning_datasets'
label_file = '/Users/mona/Documents/GitHub/group_5_DL4M1/new_data/train.txt'

In [3]:
#is there a way to not having to load data and labels everytime?
audio_data, audio_labels = u.load_data(wav_directory, label_file)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mona/Documents/GitHub/group_5_DL4M1/new_data/japanese_full_traning_datasets'

In [4]:
# Example of using loaded data
print("Loaded", len(audio_data), "audio files.")
print("Sample labels:", audio_labels[:10]) # Print the first 10 labels

NameError: name 'audio_data' is not defined

#### Japanese Data

In [ ]:
#load dev set
wav_directory = '/teamspace/studios/this_studio/data/japanese_dev/japanese_dev'
label_file = '/teamspace/studios/this_studio/data/japanese_dev/japanese_dev.txt'
u.copy_selected_flac_files(wav_directory, label_file)

In [ ]:
wav_directory = '/teamspace/studios/this_studio/data/japanese_full_train_set_flac2.0/japanese_train_set_flac'
label_file = '/teamspace/studios/this_studio/data/train.txt'
_, train_labels = u.load_flac_data(wav_directory, label_file)

In [ ]:
print(len(train_labels))

In [ ]:
np.save('/teamspace/studios/this_studio/jap_aug_train_labels.npy', train_labels)

In [ ]:
output_file = '/teamspace/studios/this_studio/jap_aug_train_labels.txt'
with open(output_file, 'w') as f:
    for label in train_labels:
        f.write(f'{label}\n')

In [ ]:
files = os.listdir('/teamspace/studios/this_studio/data/dev_bonafide_audio/japanese_full_train_set_flac/japanese_train_set_flac')
    
    # Count the number of files
num_files = len(files)
print(num_files)

In [ ]:
files = os.listdir('/teamspace/studios/this_studio/data/japanese_full_train_set_flac2.0/japanese_train_set_flac')
    
    # Count the number of files
num_files = len(files)
print(num_files)

In [ ]:
import os

# Directory containing your audio files
directory = '/teamspace/studios/this_studio/data/dev_bonafide_audio/japanese_full_train_set_flac/japanese_train_set_flac'

# Initialize a list to store bonafide labels
bonafide_labels = []

# List all files in the directory
files = os.listdir(directory)

# Iterate over the files
for file_name in files:
    # Check if the file name starts with "bonafide"
    if file_name.startswith('bonafide'):
        # Add the label "bonafide" to the bonafide_labels list
        bonafide_labels.append('bonafide')

print(len(bonafide_labels))

In [ ]:
augmented_bonafide_audio = "/teamspace/studios/this_studio/data/japanese_augmented_bonafide"
bonafide_labels = u.create_bonafide_labels(augmented_bonafide_audio)

In [ ]:
print(bonafide_labels[1:10])

In [ ]:
train_labels.extend(bonafide_labels)

In [ ]:
source_dir = "/teamspace/studios/this_studio/data/japanese_augmented_bonafide"
target_dir = "/teamspace/studios/this_studio/data/dev_bonafide_audio/japanese_full_train_set_flac/japanese_train_set_flac"

# Append directories and labels
appended_labels = u.append_directories(source_dir, target_dir, bonafide_labels, train_labels)
print(appended_labels[3000:3010])

In [ ]:
u.plot_label_distribution(dev_labels)

In [ ]:
print(len(train_labels))

In [ ]:
files = os.listdir('/teamspace/studios/this_studio/data/dev_bonafide_audio/japanese_full_train_set_flac/japanese_train_set_flac')
num_files = len(files)
print(num_files)

In [ ]:
print(len(train_labels))

In [ ]:
print("Loaded", len(train_audio_data), "audio files.")
print("Sample labels:", train_labels[300:310]) # Print the first 10 labels

In [ ]:
u.plot_label_distribution(concatenated_labels)

In [ ]:
print("Loaded", len(concatenated_labels), "labels.")
print("Sample labels:", concatenated_labels[5000:5050])

In [ ]:
import csv
def save_data_and_labels(concatenated_data, concatenated_labels, data_file, labels_file):
    # Write concatenated data to CSV
    with open(data_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(concatenated_data)

    # Write concatenated labels to CSV
    with open(labels_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(concatenated_labels)

In [ ]:
source_dir = "/teamspace/studios/this_studio/data/japanese_augmented_bonafide"
target_dir = "/teamspace/studios/this_studio/data/dev_bonafide_audio/japanese_full_train_set_flac/japanese_train_set_flac"
labels_file = "/path/to/labels.txt"

# Append audio files and generate labels
append_audio_files(source_dir, target_dir, labels_file)

In [ ]:
output_directory = "/teamspace/studios/this_studio/data/japanese_augmented_bonafide"
concatenated_data, concatenated_labels = u.augment_and_concatenate_bonafide(train_audio_data, train_labels, output_directory)

In [ ]:
dev_data = '/teamspace/studios/this_studio/data/japanese_dev/japanese_dev'
dev_labels = '/teamspace/studios/this_studio/data/japanese_dev/japanese_dev.txt'
dev_audio_data, dev_labels = u.load_flac_data(dev_data, dev_labels)


In [ ]:
print("Loaded", len(dev_audio_data), "audio files.")
print("Sample labels:", dev_labels[300:310]) # Print the first 10 labels

In [ ]:
val_audio_data, val_labels, test_audio_data, test_labels = u.test_val_split(dev_audio_data, dev_labels, test_size = 0.5)

In [ ]:
u.plot_label_distribution(val_labels)

Look at one sample from the dataset

In [ ]:
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio
import numpy as np

# Load an audio file
file_path = '/Volumes/T7/group5_DL4M/dev_set/wav_files/CtrSVDD_0059_D_0000530.wav'
audio, sr = librosa.load(file_path, sr=None)  # Load with the original sample rate

# Play the audio
display(Audio(data=audio, rate=sr))

# Plot the spectrogram. refer: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.colorbar.html
plt.figure(figsize=(10, 4))
S = librosa.feature.melspectrogram(y=audio, sr=sr)
S_dB = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram')
plt.tight_layout()
plt.show()

print("sr=", sr)


Define Splite

In [ ]:
train_data, val_data, test_data, train_labels, val_labels, test_labels = u.split_data(
    audio_data, audio_labels, test_size=0.2, val_size=0.2)

print("Training data size:", len(train_data))
print("Validation data size:", len(val_data))
print("Test data size:", len(test_data))

In [ ]:
u.plot_label_distribution(audio_labels)

In [ ]:
u.plot_audio_length_distribution(audio_data, sr)

### Data Preprocessing

MFCC & LFCC 

In [ ]:
#look at the lfcc of one audio file
audio, sr = librosa.load('/Users/mona/Documents/GitHub/group_5_DL4M1/new_data/japanese_full_traning_dataset/._CtrSVDD_0001_T_0006771.flac', sr=None)  # Load an audio file
mfccs = u.compute_mfcc(audio, sr)
print("MFCCs:", mfccs.shape)

In [ ]:
u.plot_mfcc(mfccs, sr)

In [ ]:
lfccs = u.compute_lfcc(audio, sr)
print("LFCCs:", lfccs.shape)

In [ ]:
u.plot_lfcc(lfccs, sr)

Prepare Data for Modeling

In [ ]:
# Batch Processing for Feature Extraction
# Test 
test_file_paths = ['/Users/mona/Documents/GitHub/group_5_DL4M1/new_data/japanese_full_traning_dataset/._CtrSVDD_0001_T_0006771.flac',
                   '/Users/mona/Documents/GitHub/group_5_DL4M1/new_data/japanese_full_traning_dataset/._CtrSVDD_0001_T_0006772.flac']  # Add your actual paths
test_features, test_sr = u.load_and_extract_features(test_file_paths, feature_type='lfcc')

print("Shape of extracted features:", test_features.shape)
print("Sample rate used:", test_sr)

In [ ]:
#test storing features in h5 file
import h5py

# Create an HDF5 file
with h5py.File('/Users/mona/Documents/GitHub/group_5_DL4M1/new_data/jap_LFCC_features_test.h5', 'w') as h5f:
    # Create a dataset in the file
    h5f.create_dataset('LFCC_dataset_test', data=np.array(test_features))

In [ ]:
test_features, test_sr = u.load_and_extract_features(test_file_paths, feature_type='mfcc')

print("Shape of extracted features:", test_features.shape)
print("Sample rate used:", test_sr)

In [ ]:
# from concurrent.futures import ProcessPoolExecutor

def process_file(file_path):
    features, _ = u.load_and_extract_features([file_path], feature_type='lfcc')
    return features

In [ ]:
dataset_directory = '/Users/mona/Documents/GitHub/group_5_DL4M1/new_data/japanese_full_traning_dataset'

#refer to ChatGPT
audio_files = [os.path.join(dataset_directory, f) for f in os.listdir(dataset_directory) if f.endswith('.wav') and os.path.isfile(os.path.join(dataset_directory, f))]

# with ProcessPoolExecutor() as executor:
#     all_features = list(executor.map(process_file, audio_files))

# alternative if not using parallel processing

all_features = []
for file_path in audio_files:
    feature = process_file(file_path)
    if feature is not None:
        all_features.append(feature)

# print("Features extracted. Number of files processed:", len(all_features))

# Optionally filter out None values and check shapes
valid_features = []
shapes = set()
for feature in all_features:
    valid_features.append(feature)
    shapes.add(feature.shape)

print("Unique feature shapes:", shapes)

In [ ]:
def check_corrupted_files(directory):
    corrupted_files = []

    # List all FLAC files in the directory
    flac_files = [f for f in os.listdir(directory) if f.endswith('.flac')]

    for file in flac_files:
        file_path = os.path.join(directory, file)
        try:
            # Try to load the file
            sf.read(file_path)
        except Exception as e:
            # If an exception occurs, the file is likely corrupted
            print(f"Error loading {file}: {e}")
            corrupted_files.append(file)

    if corrupted_files:
        print("Corrupted files:")
        for file in corrupted_files:
            print(file)
    else:
        print("No corrupted files found.")

# Example usage:
check_corrupted_files('/teamspace/studios/this_studio/data/japanese_augmented_bonafide')

In [ ]:
#where kernel often crashes
# Determine the maximum length from the shapes printed
max_length = 0
for shape in shapes:
    length = shape[1]
    if length > max_length:

        max_length = length\

# Pad all features to the maximum length
uniform_features = u.pad_features(all_features, max_length)

# Convert list of arrays into a single numpy array for storage
uniform_features_array = np.array(uniform_features)

In [ ]:
# Create an HDF5 file
with h5py.File('/Users/mona/Documents/GitHub/group_5_DL4M1/new_data/jap_LFCC_features_test.h5', 'w') as h5f:
    # Create a dataset in the file
    h5f.create_dataset('jap_LFCC_dataset_test', data=np.array(uniform_features_array))


In [ ]:
print("Shape of the dataset:", uniform_features_array.shape)

In [ ]:
train_data_feature, val_data_feature, test_data_feature, train_labels, val_labels, test_labels = u.split_data(
    uniform_features_array, audio_labels, test_size=0.2, val_size=0.2)

print("Training data size:", len(train_data_feature))
print("Validation data size:", len(val_data_feature))
print("Test data size:", len(test_data_feature))

### Model Training

In [ ]:
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

In [ ]:
import tensorflow as tf
train_labels_reshaped = tf.reshape(train_labels, [train_labels.shape[0], 1, 1])
val_labels_reshaped = tf.reshape(val_labels, [val_labels.shape[0], 1, 1])

In [ ]:
train_data_feature = train_data_feature[..., np.newaxis]
val_data_feature = val_data_feature[..., np.newaxis]

In [ ]:
train_data_feature_1 = np.array(train_data_feature, dtype=float)
val_data_feature_1 = np.array(val_data_feature, dtype=float)

In [ ]:
train_labels = np.where(train_labels == 'deepfake', 1, 0)
val_labels = np.where(val_labels == 'deepfake', 1, 0)
test_labels = np.where(test_labels == 'deepfake', 1, 0)

In [ ]:
print("Train data shape:", train_data_feature_1.shape)
print("Train labels shape:", train_labels.shape)
print("Validation data shape:", val_data_feature_1.shape)
print("Validation labels shape:", val_labels.shape)

In [ ]:
print("Feature array shape:", uniform_features_array.shape)

input_shape = uniform_features_array.shape  
print("Input shape for model:", input_shape)

In [ ]:
input_shape = uniform_features_array.shape 
from models import build_model
model = build_model(input_shape)
model.summary()

In [ ]:
import keras
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=f"checkpoints/audio_convnet.keras",
        save_best_only=True,
        monitor="val_loss")
]

batch_size = 32
history = model.fit(
        x=train_data_feature_1,
        y=train_labels, 
        validation_data=(val_data_feature_1, val_labels),
        epochs=10,
        batch_size=batch_size,
        callbacks=callbacks
    )

#### Check loss and accuracy

In [ ]:
u.plot_loss(history)

In [ ]:
# Load the best checkpoint of the model
aug_reloaded = keras.models.load_model("checkpoints/audio_aug.keras")

# Evaluate the model on the test set
test_loss_aug, test_acc_aug = aug_reloaded.evaluate(test_data_feature.batch(batch_size))
print(f"Test Loss: {test_loss_aug:.4f}, Test Accuracy: {test_acc_aug:.4f}")

### Transfer Learning

#### YAMNet

In [ ]:
import tensorflow_hub as hub

yamnet = hub.load('https://tfhub.dev/google/yamnet/1')

In [ ]:
from models import tcn_model

input_shape =  uniform_features_array.shape
# Create a tcn model that processes the embeddings
tcn_yamnet = tcn_model(input_shape)

# Print model summary
tcn_yamnet.summary()

In [ ]:
# Define a mapping function to extract embeddings
def map_function(audio, label):
   return u.extract_yamnet_embedding(audio, yamnet), label

# Check input shape from example in the data
for e, l in train_data_feature.map(map_function).take(1):
    print(e.shape)

In [ ]:
batch_size = 32
model_path = "checkpoints/yamnet_model.json"
model_weights = "checkpoints/yamnet_weights.h5"

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath=model_weights,
      save_best_only=True,
      save_weights_only=True,
      monitor="val_loss")
]

history = tcn_yamnet.fit(train_data_feature_1.prefetch(tf.data.AUTOTUNE).map(map_function).batch(batch_size).cache(),
    validation_data=val_data_feature_1.prefetch(tf.data.AUTOTUNE).map(map_function).batch(batch_size).cache(),
    epochs=10,
    callbacks=callbacks)

model_as_json = tcn_yamnet.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_as_json)

In [ ]:
u.plot_loss(history)

In [ ]:
# Load the best checkpoint of the model 
optimizer = 'adam'
loss = 'sparse_categorical_crossentropy'
metrics = ["accuracy"]
tcn_yamnet_reloaded = u.reload_tcn(model_path, model_weights, optimizer, loss, metrics)

# Evaluate the model on the test set
test_loss_yamnet, test_acc_yamnet = tcn_yamnet_reloaded.evaluate(test_data_feature.prefetch(tf.data.AUTOTUNE).map(map_function).batch(batch_size))
print(f"Test Loss: {test_loss_yamnet:.4f}, Test Accuracy: {test_acc_yamnet:.4f}")